In [368]:
import numpy as np
from numpy import transpose
from numpy.linalg import inv, det
from scipy.stats import norm

In [34]:
def eta(T):
    """ Generates the cutoff probabilities for exploration rounds in interval chaining. """
    return np.array([pow(t, -1/3) for t in range(1,T+1)])

In [257]:
def beta(k, d, c):
    """ Generates the scaled down feature weights for a true model. """
    return np.random.uniform(0, c+1, size=(k, d))

In [512]:
def top_interval(c, k, d, _delta, T):
    """
    Simulates T rounds of TopInterval for k
    """
    X = np.random.uniform(0, 1, size=(k, T, d))  # 3-axis ndarray
#     X = np.random.normal(0.5, 0.5, (k, T, d))
    _eta = eta(T)                             # exploration cutoff probabilities
    B = beta(k, d, c)                         # true parameters. B[i]: params for arm i
    Y = np.array([X[i].dot(transpose(B[i])) for i in range(k)])  # not sure if there's a cleaner way to do this
    picks = []
    for t in range(T):
        print('Iteration [{0} / {1}]'.format(t, T))
        if np.random.rand() <= _eta[t]:
            # Play uniformly at random from [1, k].
            picks.append(np.random.randint(0,k))
            print('Exploration round.')
        else:
            intervals = []
            for i in range(k):
                # Compute beta hat.
                _Xti = X[i][:t+1]
                _XtiT = transpose(_Xti)
                try:
                    _XTX = inv(_XtiT.dot(_Xti))
                except:
                    print('Encountered singular matrix. Ignoring.')
                    continue
                _Yti = Y[i][:t+1]
                Bh_t_i = _XTX.dot(_XtiT).dot(_Yti)  # Compute OLS estimators.
                yh_t_i = Bh_t_i.dot(X[i][t])
                _s2 = np.var(Y[i][:t+1])
                w_t_i = norm.ppf(1 - _delta/(2*T*k), loc=0, 
                                 scale=np.sqrt(_s2 * X[i][t].dot(_XTX).dot(transpose(X[i][t]))))
                intervals.append([yh_t_i - w_t_i, yh_t_i + w_t_i])
            # Pick the agent with the largest upper bound.
            picks.append(np.argmax(np.array(intervals)[:,1]) if intervals else np.random.randint(0,k))
            print('Intervals: {0}'.format(intervals))
    # Compute sum of best picks over each iteration.
    best = [transpose(Y)[i].max() for i in range(2, T)]
    performance = [Y[picks[t-2]][t] for t in range(2, T)]
#     print(picks)
#     print('Best: {0}'.format(sum(best)))
#     print('Algorithm: {0}'.format(sum(performance)))
    print('Cumulative Regret: {0}'.format(sum(best) - sum(performance)))
    print('Final Regret: {0}'.format(best[-1] - performance[-1]))
            

In [562]:
def compute_chain(i_st, intervals, k):
    # Sort intervals by decreasing order.
    chain = [i_st]
    print(intervals[:,1])
    ordering = np.argsort(intervals[:,1])[::-1]
    intervals = intervals[ordering,:]
    
    lowest_in_chain = intervals[0][0]
    for i in range(1, k):
        if intervals[i][1] >= lowest_in_chain:
            chain.append(i)
            lowest_in_chain = min(lowest_in_chain, intervals[i][0])
        else:
            return chain
    return chain

In [573]:
def interval_chaining(c, k, d, _delta, T):
    """
    Simulates T rounds of interval chaining.
    """
    X = np.random.uniform(0, 1, size=(k, T, d))  # 3-axis ndarray
#     X = np.random.normal(0.5, 0.5, (k, T, d))
    _eta = eta(T)                             # exploration cutoff probabilities
    B = beta(k, d, c)                         # true parameters. B[i]: params for arm i
    Y = np.array([X[i].dot(transpose(B[i])) for i in range(k)])  # not sure if there's a cleaner way to do this
    picks = []
    for t in range(T):
        print('Iteration [{0} / {1}]'.format(t, T))
        if np.random.rand() <= _eta[t]:
            # Play uniformly at random from [1, k].
            picks.append(np.random.randint(0,k))
            print('Exploration round.')
        else:
            intervals = []
            for i in range(k):
                # Compute beta hat.
                _Xti = X[i][:t+1]
                _XtiT = transpose(_Xti)
                try:
                    _XTX = inv(_XtiT.dot(_Xti))
                    _Yti = Y[i][:t+1]
                    Bh_t_i = _XTX.dot(_XtiT).dot(_Yti)  # Compute OLS estimators.
                    yh_t_i = Bh_t_i.dot(X[i][t])
                    _s2 = np.var(Y[i][:t+1])
                    w_t_i = norm.ppf(1 - _delta/(2*T*k), loc=0, 
                                 scale=np.sqrt(_s2 * X[i][t].dot(_XTX).dot(transpose(X[i][t]))))
                    intervals.append([yh_t_i - w_t_i, yh_t_i + w_t_i])
                except:
                    print('Encountered singular matrix. Defaulting to exploration round.')
                    intervals = None
                    break
            # Pick a random uniformly at random from the chain containing the highest quality individual.
            if not intervals:
                picks.append(np.random.randint(0,k))
            else:
                i_st = np.argmax(np.array(intervals)[:,1])
                chain = compute_chain(i_st, np.array(intervals), k)
                print('Computed chain: {0}'.format(chain))
                picks.append(np.random.choice(chain))
            print('Intervals: {0}'.format(intervals))
    # Compute sum of best picks over each iteration.
    best = [transpose(Y)[i].max() for i in range(2, T)]
    performance = [Y[picks[t-2]][t] for t in range(2, T)]
#     print(picks)
#     print('Best: {0}'.format(sum(best)))
#     print('Algorithm: {0}'.format(sum(performance)))
    print('Cumulative Regret: {0}'.format(sum(best) - sum(performance)))
    print('Final Regret: {0}'.format(best[-1] - performance[-1]))
            

In [572]:
interval_chaining(c=10, k=2, d=10, _delta=0.05, T=1000)

Iteration [0 / 1000]
Exploration round.
Iteration [1 / 1000]
Encountered singular matrix. Defaulting to exploration round.
Intervals: None
Iteration [2 / 1000]
Exploration round.
Iteration [3 / 1000]
Exploration round.
Iteration [4 / 1000]
Exploration round.
Iteration [5 / 1000]
Exploration round.
Iteration [6 / 1000]
Exploration round.
Iteration [7 / 1000]
Exploration round.
Iteration [8 / 1000]
[ 50.82916792  70.64877427]
Computed chain: [1, 1]
Intervals: [[-27.374290718082605, 50.829167920590564], [18.915647169463888, 70.648774265756771]]
Iteration [9 / 1000]
Exploration round.
Iteration [10 / 1000]
[ 69.47979899  49.3034366 ]
Computed chain: [0, 1]
Intervals: [[3.6223887793180225, 69.47979898793659], [5.4140659958561486, 49.3034365950124]]
Iteration [11 / 1000]
Exploration round.
Iteration [12 / 1000]
[ 59.8592993   52.46071655]
Computed chain: [0, 1]
Intervals: [[0.84451923959159458, 59.859299304691632], [11.929656406326341, 52.460716552231958]]
Iteration [13 / 1000]
Exploration r

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in sqrt



Computed chain: [0]
Intervals: [[35.319238831316518, 48.953460617362914], [16.266699809134689, 30.31430422012988]]
Iteration [119 / 1000]
[ 46.01189346  28.09769891]
Computed chain: [0]
Intervals: [[31.774089832920826, 46.011893461623252], [17.266708677890833, 28.097698910187873]]
Iteration [120 / 1000]
[ 40.10031421  29.21331376]
Computed chain: [0, 1]
Intervals: [[26.220709255713128, 40.100314211689494], [17.017942461284775, 29.213313762673224]]
Iteration [121 / 1000]
[ 40.14151908  32.2318037 ]
Computed chain: [0, 1]
Intervals: [[26.003394101213157, 40.141519080089957], [19.956642948756556, 32.231803696249031]]
Iteration [122 / 1000]
[ 37.8721838   22.20742972]
Computed chain: [0, 1]
Intervals: [[21.130644288378701, 37.872183803908285], [9.8512190741458987, 22.207429715937035]]
Iteration [123 / 1000]
[ 38.09700537  33.19178886]
Computed chain: [0, 1]
Intervals: [[24.703427178317337, 38.097005366420198], [20.715757120092107, 33.191788858749554]]
Iteration [124 / 1000]
[ 37.97998955 

### Testing

In [359]:
tmp = np.array([[ 0.41617394,  0.54593447,  0.10577638,  0.66642839,  0.21852587,
         0.12408503,  0.19740174,  0.16025822,  0.34112945,  0.79106745],
       [ 0.09355889,  0.68958286,  0.77347255,  0.02070936,  0.07084052,
         0.5991455 ,  0.45762994,  0.50857438,  0.17216335,  0.11820663]])

In [362]:
transpose(tmp).dot(tmp)

array([[ 0.18195401,  0.29172031,  0.11638661,  0.27928767,  0.09757253,
         0.10769634,  0.12496881,  0.11427695,  0.1580766 ,  0.34028094],
       [ 0.29172031,  0.77356897,  0.59112039,  0.37810705,  0.16815121,
         0.48090276,  0.42334218,  0.43819466,  0.30495522,  0.51338426],
       [ 0.11638661,  0.59112039,  0.60944843,  0.0865105 ,  0.07790807,
         0.47654786,  0.37484464,  0.41031986,  0.16924706,  0.17510583],
       [ 0.27928767,  0.37810705,  0.0865105 ,  0.44455568,  0.14709891,
         0.09510171,  0.14103135,  0.11733288,  0.23090374,  0.52963779],
       [ 0.09757253,  0.16815121,  0.07790807,  0.14709891,  0.05277194,
         0.06955957,  0.07555613,  0.07104824,  0.08674175,  0.18124252],
       [ 0.10769634,  0.48090276,  0.47654786,  0.09510171,  0.06955957,
         0.37437242,  0.29868152,  0.3245957 ,  0.14547995,  0.1689826 ],
       [ 0.12496881,  0.42334218,  0.37484464,  0.14103135,  0.07555613,
         0.29868152,  0.24839261,  0.26437411

In [191]:
k = 5
d = 10
T = 50
B = beta(k, d)
X = np.random.uniform(size=(k, T, d))
Y = np.array([X[i].dot(transpose(B[i])) for i in range(k)])

In [156]:
s2 = np.var(Y[0])
s2

0.54880669730882059

In [192]:
xTx = transpose(X[0][:10]).dot(X[0][:10])
Bh = inv(xTx).dot(transpose(X[0][:10])).dot(Y[0][:10])